In [ ]:
# ercot_volatility_analysis.py (GARCH-X – Single Regressor: Solar Only, Full Period + Summary Visuals)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Prepare output ===
results = []
summary_labels = []

for zone in zones:
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'solar_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan)
    df['solar_std'] = df['solar_mw'].rolling(72).std().bfill()
    df = df.dropna(subset=['log_return', 'solar_std'])

    y = df['log_return'] * 100
    x = df[['solar_std']]
    x = (x - x.mean()) / x.std()

    try:
        model = arch_model(y, vol='Garch', p=1, q=1, x=x)
        res = model.fit(disp='off')
        coef = res.params.get('x[0]', np.nan)
        pval = res.pvalues.get('x[0]', np.nan)
        sig_label = (
            "✅ Strong effect" if pval < 0.01 else
            "🔸 Weak effect" if pval < 0.1 else
            "❌ Not significant"
        )
        summary_labels.append(f"{zone}: {round(coef, 2)} ({sig_label})")

        print(f"\n[{zone} – solar_std]\n")
        print(res.summary())

    except Exception as e:
        coef = np.nan
        pval = np.nan
        summary_labels.append(f"{zone}: model failed")
        print(f"Model failed for {zone}: {e}")

    results.append({
        'Zone': zone,
        'Coef': round(coef, 4) if not pd.isna(coef) else np.nan,
        'PVal': round(pval, 4) if not pd.isna(pval) else np.nan
    })

# === Display Results ===
result_df = pd.DataFrame(results)
print("\n=== GARCH-X Solar Coefficients (Full Period) ===")
print(result_df.set_index('Zone'))

# === Plot: Summary Bar Chart with Annotations ===
fig = go.Figure()
fig.add_trace(go.Bar(
    x=result_df['Zone'],
    y=result_df['Coef'],
    marker_color='darkorange',
    text=summary_labels,
    textposition="outside"
))
fig.update_layout(
    title="GARCH-X Coefficients for Solar – Full Period",
    xaxis_title="Zone",
    yaxis_title="Coefficient Value",
    height=500,
    margin=dict(l=60, r=60, t=60, b=60)
)
fig.show()


In [ ]:
# ercot_volatility_analysis.py (GARCH-X – Single Regressor: Solar Only, Full Period + Summary Visuals + Volatility Trend Plots)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Prepare output ===
results = []
summary_labels = []
vol_trends = {}

for zone in zones:
    price_col = f'{zone}_rtm'
    df = full_df[['datetime', price_col, 'solar_mw']].copy()
    df = df[df[price_col] > 0]
    df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan)
    df['solar_std'] = df['solar_mw'].rolling(72).std().bfill()
    df = df.dropna(subset=['log_return', 'solar_std'])

    y = df['log_return'] * 100
    x = df[['solar_std']]
    x = (x - x.mean()) / x.std()

    try:
        model = arch_model(y, vol='Garch', p=1, q=1, x=x)
        res = model.fit(disp='off')
        coef = res.params.get('x[0]', np.nan)
        pval = res.pvalues.get('x[0]', np.nan)
        cond_vol = res.conditional_volatility

        df['cond_vol'] = cond_vol
        vol_trends[zone] = df[['datetime', 'cond_vol', 'solar_std']].copy()

        sig_label = (
            "✅ Strong effect" if pval < 0.01 else
            "🔸 Weak effect" if pval < 0.1 else
            "❌ Not significant"
        )
        summary_labels.append(f"{zone}: {round(coef, 2)} ({sig_label})")

        print(f"\n[{zone} – solar_std]\n")
        print(res.summary())

    except Exception as e:
        coef = np.nan
        pval = np.nan
        summary_labels.append(f"{zone}: model failed")
        print(f"Model failed for {zone}: {e}")

    results.append({
        'Zone': zone,
        'Coef': round(coef, 4) if not pd.isna(coef) else np.nan,
        'PVal': round(pval, 4) if not pd.isna(pval) else np.nan
    })

# === Display Results ===
result_df = pd.DataFrame(results)
print("\n=== GARCH-X Solar Coefficients (Full Period) ===")
print(result_df.set_index('Zone'))

# === Plot: Summary Bar Chart with Annotations ===
fig = go.Figure()
fig.add_trace(go.Bar(
    x=result_df['Zone'],
    y=result_df['Coef'],
    marker_color='darkorange',
    text=summary_labels,
    textposition="outside"
))
fig.update_layout(
    title="GARCH-X Coefficients for Solar – Full Period",
    xaxis_title="Zone",
    yaxis_title="Coefficient Value",
    height=500,
    margin=dict(l=60, r=60, t=60, b=60)
)
fig.show()

# === Plot: Time Series of Volatility vs. Solar_STD ===
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8), sharex=True)
axes = axes.flatten()

for i, zone in enumerate(zones):
    ax = axes[i]
    if zone in vol_trends:
        zdf = vol_trends[zone]
        ax.plot(zdf['datetime'], zdf['cond_vol'], label='Volatility (GARCH)', color='navy')
        ax.plot(zdf['datetime'], zdf['solar_std'], label='Solar Std Dev', color='orange', alpha=0.6)
        ax.set_title(zone)
        ax.set_ylabel("Scaled Values")
        ax.legend(loc='upper right')

fig.suptitle("Solar Variability vs. Conditional Volatility – GARCH-X Trend", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# ercot_volatility_analysis.py (GARCH-X – Single Regressor: Wind + BESS, Full Period)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns

# === Load dataset ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
full_df = pd.read_csv(file_path, parse_dates=['datetime'])

# === Define zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Loop over regressors: wind and bess ===
for regressor in ['wind_mw', 'net_output_mw']:
    reg_label = 'Wind' if regressor == 'wind_mw' else 'BESS'
    reg_std = regressor.replace('_mw', '_std')

    results = []
    summary_labels = []
    vol_trends = {}

    for zone in zones:
        price_col = f'{zone}_rtm'
        df = full_df[['datetime', price_col, regressor]].copy()
        df = df[df[price_col] > 0]
        df['log_return'] = np.log(df[price_col]).diff().replace([np.inf, -np.inf], np.nan)
        df[reg_std] = df[regressor].rolling(72).std().bfill()
        df = df.dropna(subset=['log_return', reg_std])

        y = df['log_return'] * 100
        x = df[[reg_std]]
        x = (x - x.mean()) / x.std()

        try:
            model = arch_model(y, vol='Garch', p=1, q=1, x=x)
            res = model.fit(disp='off')
            coef = res.params.get('x[0]', np.nan)
            pval = res.pvalues.get('x[0]', np.nan)
            cond_vol = res.conditional_volatility

            df['cond_vol'] = cond_vol
            vol_trends[zone] = df[['datetime', 'cond_vol', reg_std]].copy()

            sig_label = (
                "✅ Strong effect" if pval < 0.01 else
                "🔸 Weak effect" if pval < 0.1 else
                "❌ Not significant"
            )
            summary_labels.append(f"{zone}: {round(coef, 2)} ({sig_label})")

            print(f"\n[{zone} – {reg_std}]")
            print(res.summary())

        except Exception as e:
            coef = np.nan
            pval = np.nan
            summary_labels.append(f"{zone}: model failed")
            print(f"Model failed for {zone}: {e}")

        results.append({
            'Zone': zone,
            'Coef': round(coef, 4) if not pd.isna(coef) else np.nan,
            'PVal': round(pval, 4) if not pd.isna(pval) else np.nan
        })

    # === Plot Coefficient Summary ===
    result_df = pd.DataFrame(results)
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=result_df['Zone'],
        y=result_df['Coef'],
        marker_color='teal' if regressor == 'wind_mw' else 'crimson',
        text=summary_labels,
        textposition="outside"
    ))
    fig.update_layout(
        title=f"GARCH-X Coefficients for {reg_label} – Full Period",
        xaxis_title="Zone",
        yaxis_title="Coefficient Value",
        height=500,
        margin=dict(l=60, r=60, t=60, b=60)
    )
    fig.show()

    # === Plot Time Series Trends ===
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8), sharex=True)
    axes = axes.flatten()

    for i, zone in enumerate(zones):
        ax = axes[i]
        if zone in vol_trends:
            zdf = vol_trends[zone]
            ax.plot(zdf['datetime'], zdf['cond_vol'], label='Volatility (GARCH)', color='navy')
            ax.plot(zdf['datetime'], zdf[reg_std], label=f'{reg_label} Std Dev', alpha=0.6)
            ax.set_title(zone)
            ax.set_ylabel("Scaled Values")
            ax.legend(loc='upper right')

    fig.suptitle(f"{reg_label} Variability vs. Conditional Volatility – GARCH-X Trend", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

# === Load and preprocess ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df = pd.read_csv(file_path, parse_dates=["datetime"])

zone = "LZ_HOUSTON"
price_col = f"{zone}_rtm"

# Filter and clean
df = df[df[price_col] > 0].copy()
df["log_return"] = np.log(df[price_col]).diff().clip(-3, 3) * 100
df["wind_std"] = df["wind_mw"].rolling(24).std().bfill()
df = df.dropna(subset=["log_return", "wind_std"])

y = df["log_return"]
x = (df[["wind_std"]] - df[["wind_std"]].mean()) / df[["wind_std"]].std()

# === GARCH-X ===
print("Fitting GARCH-X...")
model_garchx = arch_model(y, vol="Garch", p=1, q=1, x=x)
res_garchx = model_garchx.fit(disp="off")
print(res_garchx.summary())

# === TGARCH (GJR-GARCH) ===
print("\nFitting TGARCH...")
model_tgarch = arch_model(y, vol="GARCH", p=1, o=1, q=1)
res_tgarch = model_tgarch.fit(disp="off")
print(res_tgarch.summary())

# === Regime Switching ===
print("\nFitting Regime-Switching...")
model_rs = MarkovRegression(y, k_regimes=2, trend="c", switching_variance=True)
res_rs = model_rs.fit()
print(res_rs.summary())

# === Model Comparison ===
print("\n=== AIC / BIC Comparison ===")
aic_bic_df = pd.DataFrame({
    "Model": ["GARCH-X", "TGARCH", "Regime Switching"],
    "AIC": [res_garchx.aic, res_tgarch.aic, res_rs.aic],
    "BIC": [res_garchx.bic, res_tgarch.bic, res_rs.bic]
})
print(aic_bic_df)

# === Plot: Regime Probabilities ===
res_rs.smoothed_marginal_probabilities[0].plot(figsize=(12, 4), title="Probability of Regime 0 (Low Volatility)")
plt.xlabel("Time")
plt.ylabel("Probability")
plt.grid(True)
plt.tight_layout()
plt.show()

# === Plot: Conditional Volatility ===
plt.figure(figsize=(12, 4))
plt.plot(res_garchx.conditional_volatility, label="GARCH-X", alpha=0.8)
plt.plot(res_tgarch.conditional_volatility, label="TGARCH", alpha=0.8)
plt.title("Conditional Volatility Comparison")
plt.ylabel("Volatility")
plt.xlabel("Time")
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()
